In this cell takes place the import of the necessary tools.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

Here is reading the dataset

In [2]:
url = "/datacov.csv"
data = pd.read_csv(url)
data.info()
print(f"Read in {len(data)} rows")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554328 entries, 0 to 554327
Columns: 173 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(95), int64(12), object(66)
memory usage: 731.6+ MB
Read in 554328 rows


In [3]:
vacines1 = data['FAB_COV_1'].unique()
vacines2 = data['FAB_COV_2'].unique()
vacines_lote = data['LAB_PR_COV'].unique()
print(vacines1)
print("\n\n\n")
print(vacines2)
print("\n\n\n")
print(vacines_lote)

['86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC'
 '87 - COVID-19 PFIZER - COMIRNATY' nan ... 'OFTAGEL' 'PHIZER 2 DOSES'
 'OXFORD / ASTRAZENICA']




[nan '86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC' 'CORONAVAC'
 '85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD' 'FIOCRUZ'
 '87 - COVID-19 PFIZER - COMIRNATY'
 '89 - COVID-19 ASTRAZENECA - CHADOX1-S' 'ASTRAZENECA/OXFORD/FIOCRUZ'
 'BUTANTAN' 'ASTRAZENECA' 'ASTRAZENICA' 'BUTANTA'
 '98 - COVID-19 SINOVAC - CORONAVAC' 'INSTITUTO BUTANTAN'
 '99 - COVID-19 PEDIATRICA - PFIZER COMIRNATY' 'FF5110' 'SINOVAC/BUTANTAN'
 '217VCD2351' 'PFIZER' '88 - COVID-19 JANSSEN - AD26.COV2.S' 'SINOVAC'
 'ASTRAZENECA FIOCRUZ' '215VCD171Z' 'OXFORD'
 'COVID-19 - CHADOX1 NCOV-19 (FIOCRUZ/OXFORD)' 'FG3533' 'OSWALDO CRUZ'
 'ASTRAZENECA/FIOCRUZ' 'COVID-19-COVISHIELD-OXFORD/FIOCRUZ'
 'SINOVAC / BUTANTAN' 'CORONAVAC,BUTANTAN'
 'COVID-19-COVISHIELD-OXFORD/FIOCRUZCOVID-19-COVISHIELD-OXFORD/FIOCRUZ'
 'PIZER' 'COVID-19 SINOVAC/BUTANTAN - CORONAVAC' 'OSFORD'
 'CORONAVAC - BUTANTAN 

Here manually droping the columns containing irrelevant and redundant data 

In [4]:
data.drop(['DT_NASC', 'TP_IDADE','CS_ESCOL_N', 'CO_PAIS', 'DT_DIGITA', 'FNT_IN_COV', 'ID_PAIS','LOTE_2_COV','FAB_COV_1', 
        'AVE_SUINO', 'POS_PCRFLU', 'POS_PCROUT', 'HISTO_VGM', 'PCR_SARS2', 'ID_REGIONA','ID_MUNICIP','FAB_COV_2','LAB_PR_COV',
        'ID_UNIDADE', 'ID_RG_RESI', 'ID_MN_RESI', 'ID_RG_INTE', 'ID_MN_INTE', 'COD_IDADE', 'LOTE_1_COV',
          ], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554328 entries, 0 to 554327
Columns: 148 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(90), int64(8), object(50)
memory usage: 625.9+ MB


Droping the lines with contain data of type 3 and 9

In [5]:
aux = data[(data['EVOLUCAO'] == 3) | (data['EVOLUCAO'] == 9)].index
data.drop(aux, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526276 entries, 0 to 554327
Columns: 148 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(90), int64(8), object(50)
memory usage: 598.3+ MB


In [6]:
x = data
x.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,NU_IDADE_N,CS_GESTANT,CS_RACA,SG_UF,CO_RG_RESI,CO_MUN_RES,CS_ZONA,SURTO_SG,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,MORB_DESC,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,OUT_ANTIV,DT_ANTIVIR,HOSPITAL,DT_INTERNA,SG_UF_INTE,CO_RG_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,OUT_AMOST,PCR_RESUL,DT_PCR,TP_FLU_PCR,PCR_FLUASU,FLUASU_OUT,PCR_FLUBLI,FLUBLI_OUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,DS_PCR_OUT,CLASSI_FIN,CLASSI_OUT,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FAB_COVREF,LOTE_REF,DOSE_2REF,FAB_COVRF2,LOTE_REF2,TRAT_COV,TIPO_TRAT,OUT_TRAT,DT_TRT_COV
0,05/01/2022,1,02/01/2022,1,RJ,NaN,330455,7101856,M,23,6,4,RJ,NaN,330455.0,NaN,NaN,NaN,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,05/01/2022,RJ,NaN,330455.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,05/01/2022,4.0,OROFARINGE,2.0,10/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,1.0,05/01/2022,07/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,25/01/2021,22/02/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04/01/2022,1,03/01/2022,1,MS,1966.0,500370,2371375,F,37,5,4,MS,1966.0,500370.0,1.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,1.0,04/01/2022,MS,1966.0,500370.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,04/01/2022,1.0,NaN,1.0,08/01/2022,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,1.0,06/01/2022,10/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,NaN,NaN,NaN,2.0,04/01/2022,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,07/06/2021,10/09/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07/01/2022,1,04/01/2022,1,GO,1791.0,521890,5584108,F,33,5,4,GO,1791.0,521890.0,1.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,07/01/2022,GO,1791.0,521890.0,2.0,NaN,NaN,3.0,NaN,NaN,NaN,1.0,07/01/2022,1.0,NaN,2.0,11/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1.0,1.0,NaN,15/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,07/01/2022,2.0,2.0,2.0,2.0,1.0,21/01/2021,17/02/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07/01/2022,1,06/01/2022,1,SP,1351.0,354990,6988938,F,16,5,1,SP,1352.0,355410.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,PROVAVEL DOENCA DE LYME,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,06/01/2022,SP,1351.0,354990.0,1.0,06/01/2022,NaN,3.0,NaN,NaN,NaN,1.0,06/01/2022,1.0,NaN,2.0,06/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1.0,1.0,09/01/2022,10/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,06/01/2022,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

keeping only state of Parana lines

In [7]:
aux = x[(x['SG_UF_NOT'] != 'PR')].index
x.drop(aux , inplace=True)
x.drop(['SG_UF_NOT','SG_UF','SG_UF_INTE'], axis=1, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51936 entries, 13 to 554312
Columns: 145 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(90), int64(8), object(47)
memory usage: 57.9+ MB
Read in 51936 rows


keeping only diagnosis of COVID19 lines

In [8]:
#mantém somente diagnóstico de COVID
aux = x[(x['CLASSI_FIN'] != 5)].index
x.drop(aux , inplace=True)
x.drop(['CLASSI_FIN'], axis=1, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Columns: 144 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(89), int64(8), object(47)
memory usage: 20.2+ MB
Read in 18222 rows


In [9]:
number_columns = x[['SEM_NOT','SEM_PRI','NU_IDADE_N','CS_GESTANT','OBES_IMC']]
x.drop(['SEM_NOT','SEM_PRI','NU_IDADE_N','CS_GESTANT','OBES_IMC'], axis=1, inplace=True)
x = x.replace(9, np.nan)
x = pd.concat([x, number_columns], axis=1)

In [10]:
x.head()

,DT_NOTIFIC,DT_SIN_PRI,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,CS_RACA,CO_RG_RESI,CO_MUN_RES,CS_ZONA,SURTO_SG,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OUT_MORBI,MORB_DESC,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,OUT_ANTIV,DT_ANTIVIR,HOSPITAL,DT_INTERNA,CO_RG_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,OUT_AMOST,PCR_RESUL,DT_PCR,TP_FLU_PCR,PCR_FLUASU,FLUASU_OUT,PCR_FLUBLI,FLUBLI_OUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,DS_PCR_OUT,CLASSI_OUT,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FAB_COVREF,LOTE_REF,DOSE_2REF,FAB_COVRF2,LOTE_REF2,TRAT_COV,TIPO_TRAT,OUT_TRAT,DT_TRT_COV,SEM_NOT,SEM_PRI,NU_IDADE_N,CS_GESTANT,OBES_IMC
62,25/01/2022,16/01/2022,1356.0,410690,15245,F,NaN,1356.0,410690.0,1.0,NaN,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,"CEFALEIA, MIALGIA",2.0,1,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,20/01/2022,1356.0,410690.0,1.0,20/01/2022,09/03/2022,2.0,6.0,NaN,NaN,1.0,21/01/2022,1.0,NaN,1.0,28/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,09/03/2022,09/03/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,6.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,25/05/2021,30/08/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,3,52,5,"41,9"
80,28/01/2022,28/01/2022,1369.0,411520,2586444,M,1.0,1369.0,411520.0,1.0,NaN,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,28/01/2022,1369.0,411520.0,2.0,NaN,NaN,3.0,6.0,NaN,NaN,1.0,28/01/2022,1.0,NaN,1.0,28/01/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,31/01/2022,17/03/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,1.0,NaN,28/01/2022,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,19/04/2021,19/07/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,66,6,NaN
99,03/02/2022,28/01/2022,1356.0,410690,15644,M,1.0,1356.0,411915.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,CONGESTAO NASAL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,02/02/2022,1356.0,410690.0,1.0,03/02/2022,06/02/2022,1.0,6.0,NaN,NaN,1.0,02/02/2022,1.0,NaN,1.0,05/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,06/02/2022,09/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,30/03/2021,23/04/2021,05/11/2021,87 - COVID-19 PFIZER - COMIRNATY,FH8025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,70,6,NaN
102,07/02/2022,04/02/2022,1374.0,411460,4053907,F,1.0,1374.0,411460.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,07/02/2022,1.0,NaN,1.0,09/02/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.

In [11]:
#remoção das colunas com 50% de dados faltantes
x.dropna(axis=1, how='any', thresh=9111, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Data columns (total 55 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  18222 non-null  object 
 1   DT_SIN_PRI  18222 non-null  object 
 2   CO_REGIONA  18222 non-null  float64
 3   CO_MUN_NOT  18222 non-null  int64  
 4   CO_UNI_NOT  18222 non-null  int64  
 5   CS_SEXO     18222 non-null  object 
 6   CS_RACA     15402 non-null  float64
 7   CO_RG_RESI  18198 non-null  float64
 8   CO_MUN_RES  18206 non-null  float64
 9   CS_ZONA     16032 non-null  float64
 10  NOSOCOMIAL  16459 non-null  float64
 11  FEBRE       14970 non-null  float64
 12  TOSSE       15632 non-null  float64
 13  GARGANTA    13596 non-null  float64
 14  DISPNEIA    15309 non-null  float64
 15  DESC_RESP   14414 non-null  float64
 16  SATURACAO   15098 non-null  float64
 17  DIARREIA    13032 non-null  float64
 18  VOMITO      13032 non-null  float64
 19  OUTRO_SIN   13287 non-n

In [12]:
#Remoção das linhas com 50% de dados faltantes
x.dropna(axis=0, thresh=38, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17033 entries, 62 to 554312
Data columns (total 55 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  17033 non-null  object 
 1   DT_SIN_PRI  17033 non-null  object 
 2   CO_REGIONA  17033 non-null  float64
 3   CO_MUN_NOT  17033 non-null  int64  
 4   CO_UNI_NOT  17033 non-null  int64  
 5   CS_SEXO     17033 non-null  object 
 6   CS_RACA     14476 non-null  float64
 7   CO_RG_RESI  17012 non-null  float64
 8   CO_MUN_RES  17017 non-null  float64
 9   CS_ZONA     15104 non-null  float64
 10  NOSOCOMIAL  15689 non-null  float64
 11  FEBRE       14664 non-null  float64
 12  TOSSE       15188 non-null  float64
 13  GARGANTA    13443 non-null  float64
 14  DISPNEIA    14994 non-null  float64
 15  DESC_RESP   14209 non-null  float64
 16  SATURACAO   14853 non-null  float64
 17  DIARREIA    12975 non-null  float64
 18  VOMITO      12966 non-null  float64
 19  OUTRO_SIN   13047 non-n

In [13]:
x.head()

,DT_NOTIFIC,DT_SIN_PRI,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,CS_RACA,CO_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,ANTIVIRAL,HOSPITAL,DT_INTERNA,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,DT_COLETA,TP_AMOSTRA,PCR_RESUL,DT_PCR,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,SEM_NOT,SEM_PRI,NU_IDADE_N,CS_GESTANT
62,25/01/2022,16/01/2022,1356.0,410690,15245,F,NaN,1356.0,410690.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,2.0,2.0,1.0,20/01/2022,1356.0,410690.0,1.0,2.0,6.0,1.0,21/01/2022,1.0,1.0,28/01/2022,1.0,2.0,09/03/2022,09/03/2022,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,25/05/2021,30/08/2021,4,3,52,5
80,28/01/2022,28/01/2022,1369.0,411520,2586444,M,1.0,1369.0,411520.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,1.0,2.0,2.0,2.0,1.0,28/01/2022,1369.0,411520.0,2.0,3.0,6.0,1.0,28/01/2022,1.0,1.0,28/01/2022,1.0,1.0,31/01/2022,17/03/2022,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,19/04/2021,19/07/2021,4,4,66,6
99,03/02/2022,28/01/2022,1356.0,410690,15644,M,1.0,1356.0,411915.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,1.0,2.0,1.0,02/02/2022,1356.0,410690.0,1.0,1.0,6.0,1.0,02/02/2022,1.0,1.0,05/02/2022,1.0,2.0,06/02/2022,09/02/2022,NaN,NaN,NaN,NaN,NaN,4.0,2.0,1.0,30/03/2021,23/04/2021,5,4,70,6
175,25/03/2022,16/02/2022,1356.0,410690,15563,F,NaN,1356.0,410690.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1,2.0,2.0,2.0,2.0,1.0,18/02/2022,1356.0,410690.0,1.0,1.0,5.0,1.0,18/02/2022,1.0,1.0,18/02/2022,1.0,1.0,18/03/2022,26/04/2022,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,21/06/2022,NaN,12,7,6,6
234,28/05/2022,26/05/2022,1374.0,412770,4056698,F,1.0,1374.0,412770.0,1.0,NaN,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2,NaN,NaN,NaN,2.0,1.0,28/05/2022,1374.0,412770.0,2.0,3.0,6.0,1.0,28/05/2022,4.0,1.0,06/06/2022,1.0,1.0,01/06/2022,06/06/2022,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,19/08/2021,21/09/2021,21,21,25,5


In [14]:
x['DT_NOTIFIC'] = pd.to_datetime(x['DT_NOTIFIC'])
x['DT_ENCERRA'] = pd.to_datetime(x['DT_ENCERRA'])
x['TEMP_T'] = (x['DT_ENCERRA'] - x['DT_NOTIFIC']).dt.days

In [15]:
x.head()

,DT_NOTIFIC,DT_SIN_PRI,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,CS_RACA,CO_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,ANTIVIRAL,HOSPITAL,DT_INTERNA,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,DT_COLETA,TP_AMOSTRA,PCR_RESUL,DT_PCR,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,SEM_NOT,SEM_PRI,NU_IDADE_N,CS_GESTANT,TEMP_T
62,2022-01-25,16/01/2022,1356.0,410690,15245,F,NaN,1356.0,410690.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,2.0,2.0,1.0,20/01/2022,1356.0,410690.0,1.0,2.0,6.0,1.0,21/01/2022,1.0,1.0,28/01/2022,1.0,2.0,09/03/2022,2022-09-03,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,25/05/2021,30/08/2021,4,3,52,5,221.0
80,2022-01-28,28/01/2022,1369.0,411520,2586444,M,1.0,1369.0,411520.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,1.0,2.0,2.0,2.0,1.0,28/01/2022,1369.0,411520.0,2.0,3.0,6.0,1.0,28/01/2022,1.0,1.0,28/01/2022,1.0,1.0,31/01/2022,2022-03-17,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,19/04/2021,19/07/2021,4,4,66,6,48.0
99,2022-03-02,28/01/2022,1356.0,410690,15644,M,1.0,1356.0,411915.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,1.0,2.0,1.0,02/02/2022,1356.0,410690.0,1.0,1.0,6.0,1.0,02/02/2022,1.0,1.0,05/02/2022,1.0,2.0,06/02/2022,2022-09-02,NaN,NaN,NaN,NaN,NaN,4.0,2.0,1.0,30/03/2021,23/04/2021,5,4,70,6,184.0
175,2022-03-25,16/02/2022,1356.0,410690,15563,F,NaN,1356.0,410690.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1,2.0,2.0,2.0,2.0,1.0,18/02/2022,1356.0,410690.0,1.0,1.0,5.0,1.0,18/02/2022,1.0,1.0,18/02/2022,1.0,1.0,18/03/2022,2022-04-26,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,21/06/2022,NaN,12,7,6,6,32.0
234,2022-05-28,26/05/2022,1374.0,412770,4056698,F,1.0,1374.0,412770.0,1.0,NaN,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2,NaN,NaN,NaN,2.0,1.0,28/05/2022,1374.0,412770.0,2.0,3.0,6.0,1.0,28/05/2022,4.0,1.0,06/06/2022,1.0,1.0,01/06/2022,2022-06-06,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,19/08/2021,21/09/2021,21,21,25,5,9.0


In [16]:
x['DT_SIN_PRI1'] = pd.to_datetime(x['DT_SIN_PRI'],infer_datetime_format=True, errors='coerce' ).apply(datetime.toordinal)
x['DT_INTERNA1'] = pd.to_datetime(x['DT_INTERNA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_COLETA1'] = pd.to_datetime(x['DT_COLETA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_PCR1'] = pd.to_datetime(x['DT_PCR'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_EVOLUCA1'] = pd.to_datetime(x['DT_EVOLUCA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DOSE_1_COV1'] = pd.to_datetime(x['DOSE_1_COV'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DOSE_2_COV1'] = pd.to_datetime(x['DOSE_2_COV'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x["CS_SEXO"] = x["CS_SEXO"].replace({"F": 0, "M": 1})
x = x.drop(['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_INTERNA', 'DT_COLETA', 'DT_PCR', 'DT_EVOLUCA', 'DT_ENCERRA', 'DOSE_1_COV', 'DOSE_2_COV'], axis=1)

In [17]:
x.head()

,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,CS_RACA,CO_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,ANTIVIRAL,HOSPITAL,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,TP_AMOSTRA,PCR_RESUL,CRITERIO,EVOLUCAO,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,SEM_NOT,SEM_PRI,NU_IDADE_N,CS_GESTANT,TEMP_T,DT_SIN_PRI1,DT_INTERNA1,DT_COLETA1,DT_PCR1,DT_EVOLUCA1,DOSE_1_COV1,DOSE_2_COV1
62,1356.0,410690,15245,0,NaN,1356.0,410690.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,2.0,2.0,1.0,1356.0,410690.0,1.0,2.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,4,3,52,5,221.0,738171,738175,738176,738183,738401,737935,738032
80,1369.0,411520,2586444,1,1.0,1369.0,411520.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,1.0,2.0,2.0,2.0,1.0,1369.0,411520.0,2.0,3.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,4,4,66,6,48.0,738183,738183,738183,738183,1,737899,737990
99,1356.0,410690,15644,1,1.0,1356.0,411915.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,1.0,2.0,1.0,1356.0,410690.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,1.0,5,4,70,6,184.0,738183,738188,738188,738191,738308,737879,737903
175,1356.0,410690,15563,0,NaN,1356.0,410690.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1,2.0,2.0,2.0,2.0,1.0,1356.0,410690.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,12,7,6,6,32.0,738202,738204,738204,738204,1,738327,1
234,1374.0,412770,4056698,0,1.0,1374.0,412770.0,1.0,NaN,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2,NaN,NaN,NaN,2.0,1.0,1374.0,412770.0,2.0,3.0,6.0,1.0,4.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,21,21,25,5,9.0,738301,738303,738303,738312,738161,738021,738054


In [18]:
dadosnumericos = x[['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N', 'CS_GESTANT', 'DT_SIN_PRI1', 'DT_INTERNA1', 'DT_COLETA1',
                   'DT_PCR1', 'DT_EVOLUCA1', 'TEMP_T', 'DOSE_1_COV1', 'DOSE_2_COV1']]

x.drop(columns=['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N', 'CS_GESTANT', 'DT_SIN_PRI1', 'DT_INTERNA1', 'DT_COLETA1', 'DT_PCR1', 'DT_EVOLUCA1', 'DOSE_1_COV1', 'DOSE_2_COV1'], inplace=True)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
nome = dadosnumericos.columns
imputer = imputer.fit(dadosnumericos)
dadosnumericos = imputer.transform(dadosnumericos)
scaler = MinMaxScaler()
dadosnumericos = scaler.fit_transform(dadosnumericos)
dadosnumericos = pd.DataFrame(dadosnumericos)
dadosnumericos.columns = nome


imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
nomes = x.columns
imputer = imputer.fit(x)
x = imputer.transform(x)
x = pd.DataFrame(x)
x.columns = nomes
x = pd.concat([x, dadosnumericos], axis=1)
x.loc[x['VACINA_COV'] == 2.0, ['DOSE_1_COV1', 'DOSE_2_COV1']] = 0

In [19]:
y = x['EVOLUCAO']
x = x.drop(columns= 'EVOLUCAO')
pd.set_option('display.max_rows', None)
x.iloc[0:20, :]

,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,CS_SEXO,CS_RACA,CO_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,ANTIVIRAL,HOSPITAL,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,TP_AMOSTRA,PCR_RESUL,CRITERIO,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,TEMP_T,SEM_NOT,SEM_PRI,NU_IDADE_N,CS_GESTANT,DT_SIN_PRI1,DT_INTERNA1,DT_COLETA1,DT_PCR1,DT_EVOLUCA1,TEMP_T,DOSE_1_COV1,DOSE_2_COV1
0,1356.0,410690.0,15245.0,0.0,1.0,1356.0,410690.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1356.0,410690.0,1.0,2.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,221.0,0.058824,0.039216,0.477064,0.555556,0.038567,0.999498,0.999499,0.999508,0.999386,0.577083,0.994699,0.999374
1,1369.0,411520.0,2586444.0,1.0,1.0,1369.0,411520.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1369.0,411520.0,2.0,3.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,48.0,0.058824,0.058824,0.605505,0.666667,0.071625,0.999508,0.999508,0.999508,0.000000,0.396875,0.994650,0.999318
2,1356.0,410690.0,15644.0,1.0,1.0,1356.0,411915.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1356.0,410690.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,184.0,0.078431,0.058824,0.642202,0.666667,0.071625,0.999515,0.999515,0.999519,0.999260,0.538542,0.994623,0.999200
3,1356.0,410690.0,15563.0,0.0,1.0,1356.0,410690.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1356.0,410690.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,32.0,0.215686,0.117647,0.055046,0.666667,0.123967,0.999537,0.999537,0.999537,0.000000,0.380208,0.995227,0.000000
4,1374.0,412770.0,4056698.0,0.0,1.0,1374.0,412770.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1374.0,412770.0,2.0,3.0,6.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,9.0,0.392157,0.392157,0.229358,0.555556,0.396694,0.999671,0.999671,0.999683,0.999061,0.356250,0.994814,0.999404
5,1356.0,410690.0,3000885.0,1.0,1.0,1356.0,410690.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1356.0,410690.0,1.0,2.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,1.0,-241.0,0.019608,0.000000,0.743119,0.666667,0.011019,0.999481,0.999481,0.999481,0.000000,0.095833,0.994576,0.999160
6,1371.0,411370.0,2781859.0,0.0,1.0,1371.0,410190.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1371.0,411370.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,1.0,2.0,2.0,-213.0,0.019608,0.000000,0.568807,0.555556,0.002755,0.999485,0.999485,0.999485,0.000000,0.125000,0.000000,0.000000
7,1356.0,410420.0,13846.0,1.0,1.0,1355.0,410950.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1356.0,410420.0,1.0,2.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,5.0,2.0,1.0,0.0,0.019608,0.000000,0.752294,0.666667,0.016529,0.999481,0.999481,0.999487,0.999341,0.346875,0.994581,0.999244
8,1356.0,410690.0,15407.0,1.0,1.0,1356.0,410690.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1356.0,410690.0,2.0,2.0,6.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,48.0,0.019608,0.019608,0.678899,0.666667,0.030303,0.999479,0.999490,0.999502,0.999384,0.396875,0.994533,0.999106
9,1364.0,410480.0,7119097.0,0.0,1.0,1364.0,410480.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1364.0,410480.0,1.0,3.0,6.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,6.0,1.0,2.0,1.0,10.0,0.039216,0.019608,0.256881,0.555556,0.030303,0.999495,0.999488,0.000000,0.000000,0.357292,0.994952,0.000000


In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5, random_state=43)

score = cross_val_score(clf, x, y, cv=10)
print(f"Accuracy of floresta aleátoria Classifier is:{score}")
print('Média: {:.2} | Desvio: {:.2}'.format(np.mean(score), np.std(score)))

Accuracy of floresta aleátoria Classifier is:[0.81983568 0.83861502 0.83685446 0.82560188 0.83323547 0.82207868
 0.83206107 0.82795068 0.83440986 0.82971227]
Média: 0.83 | Desvio: 0.0059
